# BentoML Scikit-learn Tutorial


This is a sample project demonstrating basic usage of [BentoML](https://github.com/bentoml) with
Scikit-learn.

In this project, we will train a classifier model using Scikit-learn and the Iris dataset, build
an prediction service for serving the trained model via an HTTP server, and containerize the 
model server as a docker image for production deployment.


Link to source code: https://github.com/bentoml/BentoML/tree/main/examples/quickstart

### Install Dependencies

Install required python packages:

In [1]:
!pip install -r https://raw.githubusercontent.com/bentoml/BentoML/main/examples/quickstart/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 779 kB 5.2 MB/s 
     |████████████████████████████████| 64 kB 3.1 MB/s 
     |████████████████████████████████| 182 kB 66.9 MB/s 
     |████████████████████████████████| 596 kB 43.6 MB/s 
     |████████████████████████████████| 46 kB 3.9 MB/s 
     |████████████████████████████████| 235 kB 48.9 MB/s 
     |████████████████████████████████| 135 kB 62.8 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 133 kB 11.8 MB/s 
     |████████████████████████████████| 51 kB 274 kB/s 
     |████████████████████████████████| 1.2 MB 36.6 MB/s 
     |████████████████████████████████| 57 kB 4.1 MB/s 
     |████████████████████████████████| 47 kB 4.3 MB/s 
     |████████████████████████████████| 57 kB 2.1 MB/s 
     |████████████████████████████████| 2.0 MB 40.0 MB/s 
     |████████████████████████████████| 51 kB 

##  Model Training

In [2]:
from sklearn import svm, datasets

# Load training data
iris = datasets.load_iris()
X, y = iris.data, iris.target

# Model Training
clf = svm.SVC()
clf.fit(X, y)

SVC()

Save the `clf` model instance to BentoML local model store:

In [3]:
import bentoml

bentoml.sklearn.save_model("iris_clf", clf)

Model(tag="iris_clf:za6xomq2fg6nsasc", path="/root/bentoml/models/iris_clf/za6xomq2fg6nsasc/")

Models saved can be accessed via `bentoml models` CLI command:

In [4]:
!bentoml models get iris_clf:latest

name: iris_clf                                                                  
version: za6xomq2fg6nsasc                                                       
module: bentoml.sklearn                                                         
labels: {}                                                                      
options: {}                                                                     
metadata: {}                                                                    
context:                                                                        
  framework_name: sklearn                                                       
  framework_versions:                                                           
    scikit-learn: 1.0.2                                                         
  bentoml_version: 1.0.3                                                        
  python_version: 3.7.13                                                        
signatures:                 

In [5]:
!bentoml models list

 Tag                        Module           Size      Creation Time       
 iris_clf:za6xomq2fg6nsasc  bentoml.sklearn  5.78 KiB  2022-08-12 10:30:27 


To verify that the saved model can be loaded correctly:

In [6]:
loaded_model = bentoml.sklearn.load_model("iris_clf:latest")

loaded_model.predict([[5.9, 3.0, 5.1, 1.8]])

array([2])

In BentoML, the recommended way of running ML model inference in serving is via Runner, which 
gives BentoML more flexibility in terms of how to schedule the inference computation, how to 
batch inference requests and take advantage of hardware resoureces available. Saved models can
be loaded as Runner instance as shown below:


In [7]:
# Create a Runner instance:
iris_clf_runner = bentoml.sklearn.get("iris_clf:latest").to_runner()

# Runner#init_local initializes the model in current process, this is meant for development and testing only:
iris_clf_runner.init_local()

# This should yield the same result as the loaded model:
iris_clf_runner.predict.run([[5.9, 3.0, 5.1, 1.8]])

array([2])

## Serving the model

A simple BentoML Service that serves the model saved above look like this:

In [8]:
%%writefile service.py
import numpy as np
import bentoml
from bentoml.io import NumpyNdarray

iris_clf_runner = bentoml.sklearn.get("iris_clf:latest").to_runner()

svc = bentoml.Service("iris_classifier", runners=[iris_clf_runner])

@svc.api(input=NumpyNdarray(), output=NumpyNdarray())
def classify(input_series: np.ndarray) -> np.ndarray:
    return iris_clf_runner.predict.run(input_series)


Writing service.py


Note: using `%%writefile` here because `bentoml.Service` definition must be created in its own `.py` file

Start a dev model server to test out the service defined above:

In [9]:
!bentoml serve service.py:svc --reload

2022-08-12T10:30:52+0000 [INFO] [cli] Starting development BentoServer from "service.py:svc" running on http://127.0.0.1:3000 (Press CTRL+C to quit)
2022-08-12 10:30:53 circus[171] [INFO] Loading the plugin...
2022-08-12 10:30:53 circus[171] [INFO] Endpoint: 'tcp://127.0.0.1:38411'
2022-08-12 10:30:53 circus[171] [INFO] Pub/sub: 'tcp://127.0.0.1:36895'
2022-08-12T10:30:53+0000 [INFO] [observer] Watching directories: ['/content', '/root/bentoml/models']



Open your web browser at http://127.0.0.1:3000 to view the Bento UI for sending test requests.

You may also send request with `curl` command or any HTTP client, e.g.:

```bash
curl -X POST -H "content-type: application/json" --data "[[5.9, 3, 5.1, 1.8]]" http://127.0.0.1:3000/classify
```


### Build Bento for deployment

Bento is the distribution format in BentoML which captures all the source code, model files, config
files and dependency specifications required for running the service for production deployment. Think 
of it as Docker/Container designed for machine learning models.

To begin with building Bento, create a `bentofile.yaml` under your project directory:

In [11]:
%%writefile bentofile.yaml
service: "service.py:svc"
labels:
  owner: bentoml-team
  project: gallery
include:
- "*.py"
python:
  packages:
    - scikit-learn
    - pandas

Writing bentofile.yaml


Next, run `bentoml build` from current directory to start the Bento build:

In [12]:
!bentoml build

Building BentoML service "iris_classifier:2f3stsa2fklcsasc" from build context "/content"
Packing model "iris_clf:za6xomq2fg6nsasc"
Locking PyPI package versions..
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 43.1 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 83.8 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 69.8 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 kB 29.4 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 35.2 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.7/504.7 kB 30.4 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 23.3 MB/s eta 0:00:00

██████╗░███████╗███╗░░██╗████████╗░█████╗░███╗░░░███╗██╗░░░░░
██╔══██╗██╔════╝████╗░██║╚══██╔══╝██╔══██╗████╗░████║██║░░░░░
██████╦╝█████╗░░██╔██╗██║░░░██║░░░██║░░██║██╔████╔██║██║░░░░░
██╔══██╗██╔══╝░░██║╚████║░░░██║░░░██║░░██║██║╚██╔╝██║██║░░░░░
██████╦╝███████╗██║░

A new Bento is now built and saved to local Bento store. You can view and manage it via 
`bentoml list`,`bentoml get` and `bentoml delete` CLI command.

In [13]:
!bentoml list

 Tag                     Size       Creation Time        Path                   
 iris_classifier:2f3st…  16.64 KiB  2022-08-12 10:37:58  ~/bentoml/bentos/iris… 


In [15]:
!bentoml get -h

Usage: bentoml get [OPTIONS] BENTO_TAG

  Print Bento details by providing the bento_tag

  bentoml get FraudDetector:latest
  bentoml get --output=json FraudDetector:20210709_DE14C9

Options:
  -o, --output [json|yaml|path]
  -q, --quiet                    Suppress all
                                 warnings and info
                                 logs

  --verbose, --debug             Generate debug
                                 information

  --do-not-track                 Do not send usage
                                 info

  --config PATH                  BentoML
                                 configuration
                                 YAML file to
                                 apply

  -h, --help                     Show this message
                                 and exit.


## Containerize and Deployment

Bento is designed to be deployed to run efficiently in a variety of different environments.
And there are lots of deployment options and tools as part of the BentoML eco-system, such as 
[Yatai](https://github.com/bentoml/Yatai) and [bentoctl](https://github.com/bentoml/bentoctl) for
direct deployment to cloud platforms.

In this guide, we will show you the most basic way of deploying a Bento, which is converting a Bento
into a Docker image containing the HTTP model server.

Make sure you have docker installed and docker deamon running, and run the following commnand:

```bash
bentoml containerize iris_classifier:latest
```

This will build a new docker image with all source code, model files and dependencies in place,
and ready for production deployment. To start a container with this docker image locally, run:

```bash
docker run -p 3000:3000 iris_classifier:invwzzsw7li6zckb2ie5eubhd 
```

## What's Next?

- 👉 [Pop into our Slack community!](https://l.linklyhq.com/l/ktO8) We're happy to help with any issue you face or even just to meet you and hear what you're working on.

- Dive deeper into the [Core Concepts](https://docs.bentoml.org/en/latest/concepts/index.html) in BentoML
- Learn how to use BentoML with other ML Frameworks at [Frameworks Guide](https://docs.bentoml.org/en/latest/frameworks/index.html) or check out other [gallery projects](https://github.com/bentoml/BentoML/tree/main/examples)
- Learn more about model deployment options for Bento:
  - [🦄️ Yatai](https://github.com/bentoml/Yatai): Model Deployment at scale on Kubernetes
  - [🚀 bentoctl](https://github.com/bentoml/bentoctl): Fast model deployment on any cloud platform
